# Inference Time Study

In "*/usr/local/lib/python3.10/dist-packages/mmengine/model/base_model/base_model.py*" file (mmengine) with class BaseModel add **imports** 

```python
import time
from demo.json_handler import JSONHandler
import torch
```

and **the following** (in test_step):

```python
        torch.cuda.synchronize()
        begin = time.time()
        data = self.data_preprocessor(data, False)
        torch.cuda.synchronize()
        end = time.time()

        out_file = '/home/michele/code/michele_mmdet3d/data/minerva_polimove/inference_times.json'
        handler = JSONHandler(out_file)
        handler.update_dictionary({'Pre-processing delta_t': (end-begin)})

        return self._run_forward(data, mode='predict')  # type: ignore
```

In [ ]:
#################################################################################################################
#                                      INFERENCE AND SAVE THE DATA                                              #
#################################################################################################################

# Needed imports
import sys
from json_handler import JSONHandler
import time
import torch


# Add the new paths separately
new_path2 = "/home/michele/code/michele_mmdet3d/"
if not new_path2 in sys.path:
    sys.path.insert(1, new_path2)
# Import the fucking wicked class
from mmdet3d.apis import LidarDet3DInferencer


# Initialize the actual inferencer class
inferencer = LidarDet3DInferencer(model='/home/michele/code/michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py', 
                                  weights='/home/michele/code/michele_mmdet3d/work_dirs/pointpillars_minerva/epoch_120.pth',
                                  show_progress=False)


# Read the files in validation list
val_list_txt_file = "/home/michele/code/michele_mmdet3d/data/minerva_polimove/ImageSets/val.txt"
with open(val_list_txt_file, 'r') as file:
    val_file_names = [line.strip() for line in file]
print(f"Total validation point_clouds: {len(val_file_names)}")


# Create the list of inputs, suitable for the inferencer
inputs = []
for i, name in enumerate(val_file_names):
    inputs.append(dict(points=("/home/michele/code/michele_mmdet3d/data/minerva_polimove/training/velodyne/"+name+".bin")))


# Create an instance JSONHandler
out_file = '/home/michele/code/michele_mmdet3d/data/minerva_polimove/inference_times.json'
handler = JSONHandler(out_file)
# Remove the file if already existing
handler.reset()
# Do the actual inference
results = []
for i, input in enumerate(inputs):
    begin = time.time()
    handler.add_dictionary({'Everything start': begin})
    results.append(inferencer(input))
    end = time.time()
    handler.update_dictionary({'Everything end': end})

In [ ]:
for element in handler.read_json_file():
    print(element)

dictionary_list = handler.read_json_file()
print(f"\nThere are {len(dictionary_list)} elements in total")

In [ ]:
#################################################################################################################
#                                  STUDY OF BASE_MODEL (NOT ACCESSIBLE)                                         #
#################################################################################################################

start = handler.get_parameter('Everything start')
end = handler.get_parameter('Voxel encoder start')
estimated_delta = []
for i in range(len(start)):
    estimated_delta.append( end[i]-start[i] )

compute_error = False

if compute_error:
    actual_delta = handler.get_parameter('Pre-processing delta_t')
    error = []
    error_abs = []
    for i in range(len(actual_delta)):
        error.append( actual_delta[i]-estimated_delta[i] )
        error_abs.append( abs( actual_delta[i]-estimated_delta[i] ) )

av_est_deltaT = sum(estimated_delta)/len(estimated_delta)
print(f"Average value of the estimated delta_t: {av_est_deltaT}")
if compute_error:
    print(f"Average value of the error: {sum(error)/len(error)}")
    av_abs_error = sum(error_abs)/len(error_abs)
    print(f"Average value for the absolute error: {av_abs_error}")
    av_act_deltat = sum(actual_delta)/len(actual_delta)
    print(f"The percentage of error is {av_abs_error/av_est_deltaT*100}. The average actual delta_t is {av_act_deltat}")

In [ ]:
#################################################################################################################
#                                            GENERAL STUDY OF TIME                                              #
#################################################################################################################

start_everything = handler.get_parameter('Everything start')
start_voxel = handler.get_parameter('Voxel encoder start')
delta_voxel = handler.get_parameter('Voxel encoder delta_t')
delta_middle = handler.get_parameter('Middle encoder delta_t')
delta_backbone = handler.get_parameter('Backbone delta_t')
delta_neck = handler.get_parameter('Neck delta_t')
delta_post = handler.get_parameter('Post-processing delta_t')
end_everything = handler.get_parameter('Everything end')

# Total time
delta_total = []
for i in range(len(start_everything)):
    delta_total.append(end_everything[i]-start_everything[i])
estimated_total = sum(delta_total)/len(delta_total)
print(f"Average total time: {estimated_total*1e3:.2f}ms which would be {1/estimated_total:.2f}Hz!")
# Preprocessing
delta_pre = []
for i in range(len(start_everything)):
    delta_pre.append(start_voxel[i]-start_everything[i])
estimated_pre = sum(delta_pre)/len(delta_pre)
print(f"\tAverage pre-processing time:\t{estimated_pre*1e3:.2f}ms which means\t{100*estimated_pre/estimated_total:.2f}% over the total. \
      \n\t\tUsually 10% higher than actual value, which would be {estimated_pre*0.90*1e3:.2f}ms")
# Voxel encoder
estimated_voxel = sum(delta_voxel)/len(delta_voxel)
print(f"\tAverage voxel encoder time:\t{estimated_voxel*1e3:.2f}ms which means\t{100*estimated_voxel/estimated_total:.2f}% over the total.")
# Middle encoder
estimated_middle = sum(delta_middle)/len(delta_middle)
print(f"\tAverage middle encoder time:\t{estimated_middle*1e3:.2f}ms which means\t{100*estimated_middle/estimated_total:.2f}% over the total.")
# Backbone
estimated_backbone = sum(delta_backbone)/len(delta_backbone)
print(f"\tAverage backbone time:\t\t{estimated_backbone*1e3:.2f}ms which means\t{100*estimated_backbone/estimated_total:.2f}% over the total.")
# Neck
estimated_neck = sum(delta_neck)/len(delta_neck)
print(f"\tAverage neck time:\t\t{estimated_neck*1e3:.2f}ms which means\t{100*estimated_neck/estimated_total:.2f}% over the total.")
# Post-processing
estimated_post = sum(delta_post)/len(delta_post)
print(f"\tAverage post-processing time:\t{estimated_post*1e3:.2f}ms which means\t{100*estimated_post/estimated_total:.2f}% over the total.")
# Final comment
sum_all_0 = estimated_pre + estimated_voxel + estimated_middle + estimated_backbone + estimated_neck + estimated_post
sum_all_1 = estimated_pre*0.90 + estimated_voxel + estimated_middle + estimated_backbone + estimated_neck + estimated_post
sum_inference = estimated_voxel + estimated_middle + estimated_backbone + estimated_neck
print(f"\nBy summing all the \"sub-components\" we get:\
      \n\tA total time of {sum_all_0*1e3:.2f}ms ({1/sum_all_0:.2f}Hz) which means\t{100*sum_all_0/estimated_total:.2f}% over the total of\t{estimated_total*1e3:.2f}ms\
      \nand by considering the \"real\" value for the preprocessing (9.85% less), we get:\
      \n\tA total time of {sum_all_1*1e3:.2f}ms ({1/sum_all_1:.2f}Hz) which means\t{100*sum_all_1/estimated_total:.2f}% over the total of\t{estimated_total*1e3:.2f}ms\
      \nwhile the inference (from voxel encoder to neck) accounts for:\
      \n\tA total time of {sum_inference*1e3:.2f}ms which means\t\t{100*sum_inference/estimated_total:.2f}% over the total of\t{estimated_total*1e3:.2f}ms")

In [ ]:
#################################################################################################################
#                                            PLOTTING OF FREQUENCY                                              #
#################################################################################################################

import matplotlib.pyplot as plt

def print_frequency_plot(values, title = "Frequency Plot"):
    # Calculating the average value of the given list
    average_value = sum(values) / len(values)

    # Plotting the frequency plot with a vertical red line for the average value
    plt.figure(figsize=(10, 6))
    plt.hist(values, bins=20, alpha=0.7, color='blue', edgecolor='black')
    plt.axvline(average_value, color='red', linestyle='dashed', linewidth=2, label=f'Average: {average_value:.4f}')
    plt.title(title)
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)
    plt.show()

# delta_inference = []
# for i in range(len(delta_voxel)):
#     delta_inference.append(delta_voxel[i] + delta_middle[i] + delta_backbone[i] + delta_neck[i])
print_frequency_plot(delta_post, "Postprocessing time")

In [6]:
# Finally, reset the file with the dictionary
handler.reset()

# What to do on MMDeploy

### Config file

In [7]:
_base_ = ['./voxel-detection_dynamic.py', '../../_base_/backends/tensorrt-fp16.py']
backend_config = dict(
    common_config=dict(max_workspace_size=1 << 30),
    model_inputs=[
        dict(
            input_shapes=dict(
                voxels=dict(
                    # What goes here and below?
                    #   - minimum, optimal, maximum shape of the tensor that is taken as input
                    #   - [a, b, c]
                    #       - a = number of voxels
                    #       - b = max number of points per voxel 
                    #       - c = dimension of the tensor (4=x,y,z,intensity)
                    # Because of the way the voxelization is performed, the same parameters can
                    # be used for all of the three cases (voxels, points, coors)
                    min_shape=[200, 32, 4],
                    opt_shape=[5000, 32, 4],
                    max_shape=[40000, 32, 4]),
                num_points=dict(
                    min_shape=[200],
                    opt_shape=[5000],
                    max_shape=[40000]),
                coors=dict(
                    min_shape=[200,  4],
                    opt_shape=[5000, 4],
                    max_shape=[40000, 4]
                )
            )
        )
    ]
)

# Notes:
#   - tensorrt.py           YES                 1. (strange error)
#   - tensorrt-fp16.py      YES                 1. (strange error)
#                                                   --> SOLVED by adjusting the "input_shapes" dictionary
#   - tensorrt-int8.py      does NOT work       2. (cannot find a file)
#
#   - onnxruntime.py        YES (only .onnx)
#   - onnxruntime-fp16.py   YES (only .onnx)    3. (couldnt't find a converter module) 
#                                                   --> SOLVED by installing it

### Command

```python
python3 tools/deploy.py configs/mmdet3d/voxel-detection/voxel-detection_tensorrt_dynamic-kitti-32x4.py /home/michele/code/michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py /home/michele/code/michele_mmdet3d/work_dirs/pointpillars_minerva/epoch_120.pth /home/michele/code/michele_mmdet3d/data/minerva_polimove/training/velodyne/1723235584511788288.bin --device cuda
```